In [1]:
from filo import FileSeries, DataSeries, DataSeriesReaderBase, DataViewerBase
from filo import TransformParameterBase

# NOTE: To run this notebook, PIL and numpy must be installed
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

%matplotlib tk

# Creation of an image series class

The image series will be based on image data stored in individual files :

In [2]:
folders = ['data/img1', 'data/img2']
files = FileSeries.auto(folders=folders, extension='.png')
files

FileSeries in . / ['data/img1', 'data/img2'], 20 files]

## Minimal example : load image data (*reader*)

In [3]:
class ImgReader(DataSeriesReaderBase):
    """How to read images from the image series"""

    def _read(self, num):
        filepath = self.data_series.files[num].path
        img = Image.open(filepath)
        return np.array(img)


class Images(DataSeries):
    """Class describing series of images stored in multiple files"""

    def __init__(self, files):
        self.files = files
        super().__init__(reader=ImgReader(data_series=self))

    # ntot and nums are optional but useful for representation/inspection ----

    @property
    def nums(self):
        """Iterator (sliceable) of data identifiers"""
        return range(self.ntot)

    @property
    def ntot(self):
        """Total number of data in the series"""
        return len(list(self.files))

Operate the `Images` class to read images based on their number in the file series:

In [4]:
images = Images(files=files)
images

Images, data length [20]
-- corrections: []
-- transforms: []

In [5]:
images.read()  # first image in the series

array([[ 70,  68,  74, ...,  69,  64,  55],
       [ 66,  54,  53, ...,  59,  59,  64],
       [ 68,  59,  62, ...,  62,  65,  62],
       ...,
       [ 80,  92,  73, ...,  94,  90, 104],
       [ 78,  90,  80, ...,  94,  92, 107],
       [ 88,  81,  84, ...,  87,  95, 106]], dtype=uint8)

In [6]:
images.read(num=8)

array([[ 71,  70,  76, ...,  76,  74,  64],
       [ 69,  57,  55, ...,  64,  62,  76],
       [ 66,  58,  62, ...,  65,  69,  69],
       ...,
       [ 78,  89,  77, ...,  99,  94, 106],
       [ 72,  87,  78, ...,  98,  97, 104],
       [ 81,  79,  82, ...,  91, 101, 114]], dtype=uint8)

## Add interactive tools (*viewer*)

In [7]:
class ImgViewer(DataViewerBase):

    def __init__(self, images):
        super().__init__()
        self.images = images

    def _create_figure(self):
        self.fig, ax = plt.subplots()
        # Below, list all axes created in the figure
        self.axs = ax,

    def _get_data(self, num):
        """Get data associated with specific image identifier"""
        return self.images.read(num=num)

    def _first_plot(self, data):
        """Create first plot based on data spit out by _get_data()"""
        ax, = self.axs
        self.imshow = ax.imshow(data)
        # below, list all elements updated during live view
        self.updated_artists = [self.imshow]

    def _update_plot(self, data):
        """How to update the plot on animated graphs"""
        self.imshow.set_array(data)


class Images(DataSeries):
    """Class describing series of images stored in multiple files"""

    def __init__(self, files):
        self.files = files
        super().__init__(
            reader=ImgReader(data_series=self),
            viewer=ImgViewer(images=self)
        )

    # ------------------------- Below, same as above -------------------------

    @property
    def nums(self):
        """Iterator (sliceable) of data identifiers"""
        return range(self.ntot)

    @property
    def ntot(self):
        """Total number of data in the series"""
        return len(list(self.files))


In [8]:
images = Images(files)
images

Images, data length [20]
-- corrections: []
-- transforms: []

When a viewer is defined, several inspection tools are automatically available : `show()`, `inspect()`, `animate()`

In [9]:
images.show(num=3)

(<Axes: >,)

In [10]:
images.inspect()

In [11]:
images.animate()

## Advanced : *transforms*

One can define corrections and transforms. Transforms are global processing tools that apply to all images in the image series in the same way ; Corrections are similar, but can vary from image to image. Here we show only how to define transforms.

As an example, we will show how to define a ROI (crop zone) on the image series

In [12]:
class RegionOfInterest(TransformParameterBase):

    # must be defined and will be the name of the correction attribute within
    # the image series class, e.g. images.roi
    name = 'roi'

    # All informations concerning the transform should be stored in self.data
    # which is automatically initiated as an empty dictionary

    @property
    def zone(self):
        """Define settable properties to not interact with self.data directly"""
        return self.data.get('zone')

    @zone.setter
    def zone(self, value):
        """Set value for the region of interest (x, y, w, h)"""
        self.data['zone'] = value

    def apply(self, img):
        """How to apply transform on the result of images.read()"""
        x0, y0, w, h = self.data['zone']
        return img[y0:y0 + h, x0:x0 + w]


class Images(DataSeries):
    """Class describing series of images stored in multiple files"""

    def __init__(self, files):
        self.files = files
        super().__init__(
            transforms=(RegionOfInterest(data_series=self),),
            reader=ImgReader(data_series=self),
            viewer=ImgViewer(images=self)
        )

    # ------------------------- Below, same as above -------------------------

    @property
    def nums(self):
        """Iterator (sliceable) of data identifiers"""
        return range(self.ntot)

    @property
    def ntot(self):
        """Total number of data in the series"""
        return len(list(self.files))

In [13]:
images = Images(files)
images.roi.zone = [280, 333, 50, 35]

In [14]:
images.show()

(<Axes: >,)

In [15]:
images.inspect()

In [16]:
images.roi.reset()

In [17]:
images.show()

(<Axes: >,)

In [18]:
images.inspect()